In [1]:
import pickle
from collections import defaultdict
import pandas as pd

read feedback data

In [ ]:
def read_buffered_pickle(filename='./feedback_all.pkl'):
    all_data = []
    try:
        with open(filename, 'rb') as f:
            while True:
                batch = pickle.load(f)
                all_data.extend(batch)
    except EOFError:
        pass
    return all_data

feedback = read_buffered_pickle()
print(f"Total data points loaded: {len(feedback)}")

read (command id, path) pairs

In [ ]:
with open('./command_path_mapping.pkl', 'rb') as f:
    command_path_mapping = pickle.load(f)

pandas

In [ ]:
df = pd.DataFrame(feedback)

data wrangling

In [ ]:
df['path_id'] = df['currentcommandid'].map(command_path_mapping)
df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
df_sorted = df.sort_values(by=['path_id', 'TimeStamp'], ascending=True)